# MARTin: Analysis of Repeated Measurements
The following lines of code extract measurements taken with MARTin out of the root export folder.

This Data will then be translated into an object-oriented structure for further processing.

In [1]:
import os
import json
import statistics
import pandas as pd
from datetime import datetime 
from IPython.display import display

### Class: Project
This class stores all measurements that belong to a project, in this case a project is the same as a participant.

In our specific case a session always contains ten measurements, one participant (or project) has five sessions.

The session start times have to be stored separately since they had to be extracted manually from the logfiles.
Times can be read out automatically since MARTin generates a timestamp at the point of each measurement.

In [2]:
class Project:
    def __init__(self, name, sessions = [], sessionStartTimes = []):
        self.name = name
        self.sessions = sessions
        self.sessionStartTimes = sessionStartTimes
    
    # Calculates the average time a measurement takes for each session.
    def getAverageMeasurementDurations(self):
        avgMesurementDurations = []
        for i, session in enumerate(self.sessions):
            lastMeasurementTime = datetime.strptime(self.sessionStartTimes[i], "%H:%M:%S")
            
            timeDeltas = []
            for measurement in session:
                if not measurement.excludedFlag:
                    timeDelta = datetime.strptime(measurement.timestamp, "%H:%M:%S") - lastMeasurementTime
                    timeDeltas.append(timeDelta.total_seconds())
                lastMeasurementTime = datetime.strptime(measurement.timestamp, "%H:%M:%S")
            
            
            avgMesurementDurations.append(sum(timeDeltas)/len(timeDeltas))
        return avgMesurementDurations
    
    # This calculates the standard deviation across the ten individual measurement of an individual session.
    def getMeasurementDurationsStDevs(self):
        mesurementDurationStDevs = []
        for i, session in enumerate(self.sessions):
            lastMeasurementTime = datetime.strptime(self.sessionStartTimes[i], "%H:%M:%S")
            
            timeDeltas = []
            for measurement in session:
                if(not measurement.excludedFlag):
                    timeDelta = datetime.strptime(measurement.timestamp, "%H:%M:%S") - lastMeasurementTime
                    timeDeltas.append(timeDelta.total_seconds())
                    
                lastMeasurementTime = datetime.strptime(measurement.timestamp, "%H:%M:%S")
            
            
            mesurementDurationStDevs.append(statistics.stdev(timeDeltas))
        return mesurementDurationStDevs
            
    # This calls the compare function for each individual session.
    def compareInSets(self):
        compSession = []
        avgMeasurementDurations = self.getAverageMeasurementDurations()
        mesurementDurationStDevs = self.getMeasurementDurationsStDevs()
        
        for i, session in enumerate(self.sessions):
            compSession.append(compare(session, avgMeasurementDurations[i],mesurementDurationStDevs[i]))
        
        return compSession
    
    # This calls compareInSets and calculates the average across all sets.
    def getSessionAverage(self):
        compSession = self.compareInSets()        
        return getDictAvg(compSession)
    
    # This calls the compare function across all sessions.
    def compareAcrossSets(self):
        dataset = []
        avgMeasurementDurations = self.getAverageMeasurementDurations()
        
        for session in self.sessions:
            dataset+= session
        # Here we calculate the StDev across the duration averages
        return compare(dataset, sum(avgMeasurementDurations)/len(avgMeasurementDurations), statistics.stdev(avgMeasurementDurations))

### Function: compare

This function contains the main elements of our data analysis.

Given a list of measurements this calculates the relative- and absolute standard deviations for each spot index across all elements of the list.

Finally the calculated values for each spot index will be averaged and returned as a dictionary.

In [30]:
def compare(dataset, avgMeasurementDuration, measurementDurationStDev): 
    normalized = []
    minimum = []
    maximum = []
    mean = []
    xPos = []
    yPos = []
    
    # This sets our lists to the same length as a measurement.
    for spot in dataset[0].measureData:
        normalized.append([])
        minimum.append([])
        maximum.append([])
        mean.append([])
        xPos.append([])
        yPos.append([])
    
    # This puts all spot values of the same index in the same list.
    for measurement in dataset:
        if(not measurement.excludedFlag):
            for i, spot in enumerate(measurement.measureData):
                normalized[i].append(spot.normalized_mean)
                minimum[i].append(spot.minPixelValue)
                maximum[i].append(spot.maxPixelValue)
                mean[i].append(spot.mean)
                xPos[i].append(spot.xPos)
                yPos[i].append(spot.yPos)       

    avg_normalized = []
    avg_minimum = []
    avg_maximum = []
    avg_mean = []
    avg_xPos = []
    avg_yPos = []

    median_normalized = []
    median_minimum = []
    median_maximum = []
    median_mean = []
    median_xPos = []
    median_yPos = []

    stDev_normalized = []
    stDev_minimum = []
    stDev_maximum = []
    stDev_mean = []
    stDev_xPos = []
    stDev_yPos = []

    stDev_perc_normalized = []
    stDev_perc_minimum = []
    stDev_perc_maximum = []
    stDev_perc_mean = []
    # Positional Data is only calculated in absolute values and therefore not listed.
    
    # Each spot position has its statistically relevant values calculated.
    for counter, pos in enumerate(normalized):
        avg_normalized.append(sum(pos)/len(pos))
        avg_minimum.append(sum(minimum[counter])/len(minimum[counter]))
        avg_maximum.append(sum(maximum[counter])/len(maximum[counter]))
        avg_mean.append(sum(mean[counter])/len(mean[counter]))
        avg_xPos.append(sum(xPos[counter])/len(xPos[counter]))
        avg_yPos.append(sum(yPos[counter])/len(yPos[counter]))

        median_normalized.append(statistics.median(pos))
        median_minimum.append(statistics.median(minimum[counter]))
        median_maximum.append(statistics.median(maximum[counter]))
        median_mean.append(statistics.median(mean[counter]))
        median_xPos.append(statistics.median(xPos[counter]))
        median_yPos.append(statistics.median(yPos[counter]))

        stDev_normalized.append(statistics.stdev(pos))
        stDev_minimum.append(statistics.stdev(minimum[counter]))
        stDev_maximum.append(statistics.stdev(maximum[counter]))
        stDev_mean.append(statistics.stdev(mean[counter]))
        stDev_xPos.append(statistics.stdev(xPos[counter]))
        stDev_yPos.append(statistics.stdev(yPos[counter]))

        if(avg_normalized[counter] > 0):
            stDev_perc_normalized.append(stDev_normalized[counter] / avg_normalized[counter])
        else:
            stDev_perc_normalized.append(0)

        if(avg_minimum[counter] > 0):
            stDev_perc_minimum.append(stDev_minimum[counter] / avg_minimum[counter])
        else:
            stDev_perc_minimum.append(0)

        if(avg_maximum[counter] > 0):
            stDev_perc_maximum.append(stDev_maximum[counter] / avg_maximum[counter])
        else:
            stDev_perc_maximum.append(0)

        if(avg_mean[counter] > 0):
            stDev_perc_mean.append(stDev_mean[counter] / avg_mean[counter])
        else:
            stDev_perc_mean.append(0)
    
    # Only the average of the relevant values are then put into a dictionary and returned.
    resultDict = {
    #'min_StDev': sum(stDev_minimum)/len(stDev_minimum),
    'Minimum pixel value rel. StDev [%]': sum(stDev_perc_minimum)/len(stDev_perc_minimum)*100,
    #'max_StDev': sum(stDev_maximum)/len(stDev_maximum),
    'Maximum pixel value rel. StDev [%]': sum(stDev_perc_maximum)/len(stDev_perc_maximum)*100,
    #'mean_StDev': sum(stDev_mean)/len(stDev_mean),
    'Mean pixel value rel. StDev [%]': sum(stDev_perc_mean)/len(stDev_perc_mean)*100,
    'X-coordinate StDev [px]': sum(stDev_xPos)/len(stDev_xPos),
    'Y-coordinate StDev [px]': sum(stDev_yPos)/len(stDev_yPos),
    'Average duration [sec]': avgMeasurementDuration,
    'Duration StDev [sec]': measurementDurationStDev
    }
    return resultDict

### Class: Measurement
Instances of this class store all values of an individual measurement.

In [31]:
class Measurement:
    def __init__(self, mId, timestamp, excludedFlag, measureData):
        self.mId = mId
        time = ""
        self.excludedFlag = excludedFlag
        extract = False
        
        # This extracts the measuring time from the timestamp.
        # Adimttedly this is not a very nice way of solving this, but it works for our purposes (directly instanciating into a datetime object had its issues).
        for sign in timestamp:
            if sign == ".":
                extract = False
            if extract:
                time += sign
            if sign == "T":
                extract = True
 
        self.timestamp = time       
        self.measureData = measureData
        

### Class: RawDatapoint

Tis is a value class containing all attributes of an individual spot of a measurement.

In [32]:
class RawDatapoint:
    def __init__(self, xPos, yPos, minPixelValue, maxPixelValue, mean, std_deviation, mean_minus_min, normalized_mean):
        self.xPos = xPos
        self.yPos = yPos
        self.minPixelValue = minPixelValue
        self.maxPixelValue = maxPixelValue
        self.mean = mean
        self.std_deviation = std_deviation
        self.mean_minus_min = mean_minus_min
        self.normalized_mean = normalized_mean

### Function: ReadoutMetadata
This extracts relevant information from the metadata-file of a measurement.

In [33]:
def readoutMetadata(metadataPath):
    with open(metadataPath) as metadataJson:
        metadata = json.load(metadataJson)

        mId = metadata["patient"]["id"]
        timestamp = metadata["datetime"]

    return mId, timestamp

### Function: ReadoutRawData
This extracts relevant information of a measurement and creates a list of instances of RawDataPoint containing the corresponding values.

In [34]:
def readoutRawData(rawDataPath, parametersPath):
    measureData = []
    with open(rawDataPath) as rawDataJson:
        with open(parametersPath) as parametersJson:
            rawData = json.load(rawDataJson)
            parameters = json.load(parametersJson)
            
            for i, spot in enumerate(rawData["values"]):
                xPos = parameters["spots"][i]["position"][0]
                yPos = parameters["spots"][i]["position"][1]
                minPixelValue = spot["min"]
                maxPixelValue = spot["max"]
                mean = spot["mean"]
                std_deviation = spot["std_deviation"]
                mean_minus_min = spot["mean_minus_min"]
                normalized_mean = spot["normalized_mean"]
                
                measureData.append(RawDatapoint(xPos, yPos, minPixelValue, maxPixelValue, mean, std_deviation, mean_minus_min, normalized_mean))
    
    return measureData

### Function: Readout
This function extracts all measurements out of a given directory and returns them as instances of Measurement.

In [35]:
def readout (path):
    projects = []
    sessions = []
    nMeasurementsExcluded = 0
    excludedFlag = False
    for projIndex, proj in enumerate(os.listdir(path)):

        projects.append(Project(proj))

        for date in os.listdir(os.path.join(path, proj)):
            session = []
            
            for measurement in os.listdir(os.path.join(path, proj, date)):
                if os.path.isdir(os.path.join(path, proj, date, measurement)):
                        # Excluded measurements contain significant errors that were made during the measurement.
                        if("EXCLUDED" in measurement):
                            nMeasurementsExcluded += 1
                            excludedFlag = True
                        else:
                            excludedFlag = False
                        
                        measurementDir = os.path.join(path, proj, date, measurement)

                        mId, timestamp = readoutMetadata(os.path.join(measurementDir, "metadata.json"))
                        session.append(Measurement(mId, timestamp, excludedFlag, readoutRawData(os.path.join(measurementDir,"data.json"), os.path.join(measurementDir, "parameters.json"))))
                    
                        
            sessions.append(session)
            
        projects[projIndex].sessions = sessions.copy()
        sessions.clear()
    
    print ("Number of excluded measurements: " + str(nMeasurementsExcluded))
    return projects

### Function: getDictAvg
Given a list of dictionaries with the same keys, this function calculates the average for each value of the same key and returns a dictionary containing these averages.

In [36]:
def getDictAvg(dictList):
    avgDict = dict.fromkeys(dictList[0].keys(),0)
    
    for d in dictList:
        for key in avgDict.keys():
            avgDict[key] += d[key]

    for key in avgDict.keys():
        avgDict[key] /= len(dictList)
    
    return avgDict

In [38]:
# Path of the root folder of the measurements
dPath = ""
projects = readout(dPath)

# Start times were extracted manually from the logfiles for each measuring session.
startTimes = [["19:59:07", "13:18:42", "13:35:08", "12:08:12", "11:43:06"],
                ["18:03:23", "17:37:04", "17:29:05", "19:20:30", "17:01:30"],
                ["20:11:30", "22:09:00", "14:10:00", "18:14:30", "17:03:00"],
                ["20:54:22", "20:59:24", "23:14:27", "18:06:36", "17:47:20"],
                ["20:54:35", "19:59:26", "22:14:41", "17:06:33", "16:47:04"],
                ["20:09:50", "23:03:31", "21:59:35", "23:10:41", "19:55:10"],
                ["17:39:00", "19:05:29", "22:22:58", "21:19:55", "17:36:20"],
                ["20:49:22", "22:39:30", "20:47:50", "23:37:12", "08:49:02"]]

mixxedDataset = [] # Effectively combination of all measurements into one big session.
tableData = [] # Data that will be put in the DataFrame
tableIndices = [] # Row names for our DataFrame
avgDurations = []
sessionAverages = []

for i, proj in enumerate(projects):
    proj.sessionStartTimes = startTimes[i]
    avgDurations += proj.getAverageMeasurementDurations()
    
    tableData += proj.compareInSets()
    
    sessionAvg = proj.getSessionAverage()
    tableData.append(sessionAvg)
    sessionAverages.append(sessionAvg)
    
    tableData.append(proj.compareAcrossSets())
    for session in proj.sessions:
        mixxedDataset += session
    
# This calculates the stDev across all set averages
tableData.append(getDictAvg(sessionAverages))
tableData.append(compare(mixxedDataset, sum(avgDurations) / len(avgDurations), statistics.stdev(avgDurations)))

# Quick and dirty way of naming all our data table rows.
for i in range(len(projects)):
    sampleName = "Participant "+ str(i+1)
    for j in range(5):
        tableIndices.append(sampleName + " - session "+ str(j+1))
    tableIndices.append(sampleName + " - session average")
    tableIndices.append(sampleName + " -  composite sessions")

tableIndices.append("General average")
tableIndices.append("Composite all sessions")


df = pd.DataFrame(tableData, index = tableIndices)
display(df)




Number of excluded measurements: 8


,Minimum pixel value rel. StDev [%],Maximum pixel value rel. StDev [%],Mean pixel value rel. StDev [%],X-coordinate StDev [px],Y-coordinate StDev [px],Average duration [sec],Duration StDev [sec]
Participant 1 - session 1,3.421705,0.520075,0.505082,0.532570,0.633220,52.200000,13.464356
Participant 1 - session 2,2.558949,0.350228,0.343467,0.394807,0.473409,51.900000,8.252272
Participant 1 - session 3,2.556727,0.350290,0.318691,0.373030,0.403153,42.300000,10.022752
Participant 1 - session 4,3.054457,0.523229,0.426037,0.531601,0.559711,42.500000,8.168367
Participant 1 - session 5,2.017273,0.268079,0.264743,0.311391,0.331054,40.000000,6.716481
Participant 1 - session average,2.721822,0.402380,0.371604,0.428680,0.480110,45.780000,9.324846
Participant 1 - composite sessions,3.380769,0.514903,0.472990,0.502454,0.519505,45.780000,5.808356
Participant 2 - session 1,4.105808,0.458871,0.678283,0.601960,0.627334,42.600000,4.501851
Participant 2 - session 2,4.222252,0.414833,0.637762,0.468493,0.696771,39.500000,7.230337
Participant 2 - session 3,1.507332,0.248109,0.326734,0.489051,0.373868,44.700000,19.032428
